In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import re
import ast

# raw data 로드 및 shape 확인

In [ ]:
product_shirt = pd.read_csv(f"{link}product_cate/001002_1page.csv")
product_hoodie = pd.read_csv(f"{link}product_cate/001004_1page.csv")
product_man2man = pd.read_csv(f"{link}product_cate/001005_1page.csv")
product_longs = pd.read_csv(f"{link}product_cate/001010_1page.csv")

In [ ]:
product_shirt.shape, product_hoodie.shape, product_man2man.shape, product_longs.shape

((100, 6), (100, 6), (100, 6), (100, 6))

In [ ]:
review_shirt = pd.read_csv(f"{link}product_review/001002_review_1page.csv")
review_hoodie = pd.read_csv(f"{link}product_review/001004_review_1page.csv")
review_man2man = pd.read_csv(f"{link}product_review/001005_review_1page.csv")
review_longs = pd.read_csv(f"{link}product_review/001010_review_1page.csv")

review_shirt["cate"] = "shirt"
review_hoodie["cate"] = "hoodie"
review_man2man["cate"] = "man2man"
review_longs["cate"] = "longs"

In [ ]:
review_shirt.shape, review_hoodie.shape, review_man2man.shape, review_longs.shape

((29451, 7), (29947, 7), (29517, 7), (28332, 7))

In [ ]:
size_shirt = pd.read_csv(f"{link}product_size/001002_size_1page.csv")
size_hoodie = pd.read_csv(f"{link}product_size/001004_size_1page.csv")
size_man2man = pd.read_csv(f"{link}product_size/001005_size_1page.csv")
size_longs = pd.read_csv(f"{link}product_size/001010_size_1page.csv")

In [ ]:
size_shirt.shape, size_hoodie.shape, size_man2man.shape, size_longs.shape

((387, 6), (393, 6), (389, 6), (395, 6))

# raw data 병합

In [ ]:
# 상품데이터 병합
product_merge_df = pd.concat([product_shirt, product_hoodie, product_man2man, product_longs], axis = 0).reset_index(drop = True)
product_merge_df.shape

(400, 6)

In [ ]:
# 리뷰데이터 병합
review_merge_df = pd.concat([review_shirt, review_hoodie, review_man2man, review_longs], axis = 0).reset_index(drop = True)
review_merge_df.shape

(117247, 7)

In [ ]:
# 사이즈 데이터 병합
size_merge_df = pd.concat([size_shirt, size_hoodie, size_man2man, size_longs], axis = 0).reset_index(drop = True)
size_merge_df.shape

(1564, 6)

# 리뷰 개수 300개가 안되는 상품

In [ ]:
review_group_under_300 = review_merge_df.groupby("goodsNo").count().reset_index()

In [ ]:
review_group_under_300[review_group_under_300["review"] != 300].shape

(37, 7)

In [ ]:
len(review_group_under_300[review_group_under_300["review"] != 300]["goodsNo"].tolist())

37

In [ ]:
review_group_under_300_list = review_group_under_300[review_group_under_300["review"] != 300]["goodsNo"].tolist()

# 사이즈 데이터가 없는 상품

In [ ]:
set(review_merge_df["goodsNo"].unique()) - set(size_merge_df["goodsNo"].unique())

{1838940, 2085361, 2085363, 2096580, 2313388, 3458174}

In [ ]:
len(set(review_merge_df["goodsNo"].unique()) - set(size_merge_df["goodsNo"].unique()))

6

In [ ]:
size_no_information = list(set(review_merge_df["goodsNo"].unique()) - set(size_merge_df["goodsNo"].unique()))

# 최종적으로 drop 되어야 할 상품의 개수

In [ ]:
set(review_group_under_300_list) & set(size_no_information)

{2096580, 3458174}

In [ ]:
drop_list_step1 = set(review_group_under_300_list + size_no_information)
len(drop_list_step1)

41

# step1 drop 처리 후 데이터

In [ ]:
product_drop_df = product_merge_df[~product_merge_df["goodsNo"].isin(drop_list_step1)].reset_index(drop = True)
product_drop_df.shape

(359, 6)

In [ ]:
review_drop_df = review_merge_df[~review_merge_df["goodsNo"].isin(drop_list_step1)].reset_index(drop = True)
review_drop_df.shape

(107700, 7)

In [ ]:
size_drop_df = size_merge_df[~size_merge_df["goodsNo"].isin(drop_list_step1)].reset_index(drop = True)
size_drop_df.shape

(1443, 6)

# step2 전처리 병합

In [ ]:
# 리뷰 / 사이즈 데이터에 대해 정규식 적용
review_drop_df["buy_size"] = review_drop_df["buy_size"].apply(lambda x: re.sub(r'[가-힣()\[\]{}\/]', ' ', x))
size_drop_df["Size"] = size_drop_df["Size"].apply(lambda x: re.sub(r'[가-힣()\[\]{}\/]', ' ', x))

In [ ]:
# 띄어쓰기 기준으로 검사를 위해 split
review_drop_df["buy_size"] = review_drop_df["buy_size"].apply(lambda x: x.strip().split(" "))
size_drop_df["Size"] = size_drop_df["Size"].apply(lambda x: x.strip().split(" "))

In [ ]:
def clean_list(lst):
    # 빈 문자열과 'NO'를 제거
    cleaned_list = [item for item in lst if item not in ['', 'NO']]

    # 특정 문자열 조합을 결합
    combined_list = []
    i = 0
    while i < len(cleaned_list):
        if i < len(cleaned_list) - 1 and cleaned_list[i] == "EXTRA" and cleaned_list[i + 1] == "LARGE":
            combined_list.append("EXTRALARGE")
            i += 2  # 두 항목을 하나로 합쳤으므로 두 인덱스를 건너뜀
        else:
            combined_list.append(cleaned_list[i])
            i += 1  # 다음 인덱스로 이동

    return combined_list

In [ ]:
review_drop_df["buy_size"] = review_drop_df["buy_size"].apply(lambda x: clean_list(x))
size_drop_df["Size"] = size_drop_df["Size"].apply(lambda x: clean_list(x))

In [ ]:
# 표기는 다르나 사이즈는 동일한 경우 중복제거
size_drop_df["Size"] = size_drop_df["Size"].astype(str)
size_drop_df.drop_duplicates(subset = ["goodsNo", "Size", "총장", "어깨너비", "가슴단면", "소매길이"], inplace = True)
size_drop_df["Size"] = size_drop_df["Size"].apply(ast.literal_eval)

In [ ]:
# 한 상품에 대하여 unique값을 사이즈 표의 unique와 비교하여 매칭이 되면 병합

merge_temp_list = []

for i in review_drop_df["goodsNo"].unique():
    review_goods = review_drop_df[review_drop_df["goodsNo"] == i]
    size_goods = size_drop_df[size_drop_df["goodsNo"] == i]

    for review_idx, review_row in review_goods.iterrows():
        for size_idx, size_row in size_goods.iterrows():
            # 교집합이 있을 경우 병합
            if len(set(review_row["buy_size"]) & set(size_row["Size"])) >= 1:
                merged_row = pd.concat([review_row, size_row], axis=0).reset_index(drop = True)
                merge_temp_list.append(merged_row)

In [ ]:
review_col_list = review_drop_df.columns.tolist()
size_col_list = size_drop_df.columns.tolist()
new_col = review_col_list + size_col_list
merge_temp_df = pd.DataFrame(merge_temp_list).reset_index(drop = True)

In [ ]:
merge_temp_df.columns = ['goodsNo', 'id_date', 'body_size', 'review', 'buy_size', 'type_class', 'cate',
                        'goodsNo2', 'Size', 'length', 'shoulder', 'chest', 'sleeve']

# 최종 데이터프레임 병합

In [ ]:
# 개수가 300개인 상품만 골라서 최종 병합
merge_temp_cnt = merge_temp_df.groupby("goodsNo").count().reset_index()
merge_temp_cnt300_list = merge_temp_cnt[merge_temp_cnt["review"] == 300]["goodsNo"].tolist()
total_temp_df = merge_temp_df[merge_temp_df["goodsNo"].isin(merge_temp_cnt300_list)].reset_index(drop = True)

In [ ]:
len(merge_temp_cnt["goodsNo"].unique())

359

In [ ]:
len(total_temp_df["goodsNo"].unique())

318

In [ ]:
final_merge_df = pd.merge(product_drop_df, total_temp_df, on = "goodsNo")

In [ ]:
final_merge_df

,goodsNo,brandName,brandNameEng,goodsName,imageUrl,relatedGoodsReviewScore,id_date,body_size,review,buy_size,type_class,cate,goodsNo2,Size,length,shoulder,chest,sleeve
0,2272830,스파오,SPAO,(시티보이) 오버핏 옥스포드 셔츠_SPYWE23C51,https://image.msscdn.net/thumbnails/images/goo...,96,\nLV.6 준준준0101\n2023.09.27\n\n\n\n\n\n,\n\n남성 · 174cm · 75kg\n\n,역사 스파오 셔츠는 그냥 기본으로 하나씩은 들고있어야 합니다! 다른 색갈들도 색감 ...,"[10, WHITE, XXL, 110]",사이즈 커요,shirt,2272830,"[XXL, 110]",80.00,65.0,69.0,59.0
1,2272830,스파오,SPAO,(시티보이) 오버핏 옥스포드 셔츠_SPYWE23C51,https://image.msscdn.net/thumbnails/images/goo...,96,\nLV.5 tae._.tuyu\n2023.09.05\n\n\n\n\n\n,\n\n남성 · 174cm · 52kg\n\n,"이미지 체인지를 조금 해보고싶어서,,, 항상 스트릿한 룩만 입다가 남친룩입어볼려고 ...","[51, LIGHT, BLUE, L, 100]",사이즈 보통이에요,shirt,2272830,"[L, 100]",77.00,62.0,64.0,56.5
2,2272830,스파오,SPAO,(시티보이) 오버핏 옥스포드 셔츠_SPYWE23C51,https://image.msscdn.net/thumbnails/images/goo...,96,\nLV.5 모호한대구토트백\n2023.10.07\n\n\n\n\n\n,\n\n남성 · 170cm · 68kg\n\n,스파오 셔츠는 색상별로 가서 이번이 3번째 입니다! 저는 오버한 핏을 원했어서 라지...,"[51, LIGHT, BLUE, L, 100]",사이즈 커요,shirt,2272830,"[L, 100]",77.00,62.0,64.0,56.5
3,2272830,스파오,SPAO,(시티보이) 오버핏 옥스포드 셔츠_SPYWE23C51,https://image.msscdn.net/thumbnails/images/goo...,96,\nLV.4 대세는남산아울렛\n2024.03.16\n\n\n\n\n\n,\n,색깔도 예쁘게 빠졌고 깔끔하고 단정하게 입기 좋아요!,"[59, NAVY, XL, 105]",사이즈 보통이에요,shirt,2272830,"[XL, 105]",79.00,63.5,66.5,58.0
4,2272830,스파오,SPAO,(시티보이) 오버핏 옥스포드 셔츠_SPYWE23C51,https://image.msscdn.net/thumbnails/images/goo...,96,\nLV.4 성으뜸\n2023.11.19\n\n\n\n\n\n,\n\n남성 · 175cm · 80kg\n\n,"안녕하세요.후기를 남기기에 앞서 저는 키가 175cm, 몸무게 80kg 인 30대 ...","[10, WHITE, XL, 105]",사이즈 보통이에요,shirt,2272830,"[XL, 105]",79.00,63.5,66.5,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95395,1841331,무신사 스탠다드,MUSINSA STANDARD,헤비웨이트 릴렉스드 크루 넥 긴팔 티셔츠 [블랙],https://image.msscdn.net/thumbnails/images/goo...,97,\nLV.5 으냐냐나\n2023.01.24\n\n\n\n\n\n,\n\n여성 · 164cm · 47kg\n\n,오버핏으로 입기 위해 큰 사이즈로 샀습니다. 생각보다 두껍진 않았어요. 무난한 무지...,[L],사이즈 커요,longs,1841331,[L],72.05,50.0,57.5,59.0
95396,1841331,무신사 스탠다드,MUSINSA STANDARD,헤비웨이트 릴렉스드 크루 넥 긴팔 티셔츠 [블랙],https://image.msscdn.net/thumbnails/images/goo...,97,\nLV.6 하라쿠우\n2023.01.23\n\n\n\n\n\n,\n\n남성 · 176cm · 70kg\n\n,넘 맘에 들어서 깔별로 모으고있어요. 이너로 최고임,[L],사이즈 보통이에요,longs,1841331,[L],72.05,50.0,57.5,59.0
95397,1841331,무신사 스탠다드,MUSINSA STANDARD,헤비웨이트 릴렉스드 크루 넥 긴팔 티셔츠 [블랙],https://image.msscdn.net/thumbnails/images/goo...,97,\nLV.6 LowF1\n2023.01.18\n\n\n\n\n\n,\n\n남성 · 168cm · 63kg\n\n,군더더기 없이 기본에 충실한 아이템입니다 굳굳,[M],사이즈 보통이에요,longs,1841331,[M],70.55,48.5,55.0,58.0
95398,1841331,무신사 스탠다드,MUSINSA STANDARD,헤비웨이트 릴렉스드 크루 넥 긴팔 티셔츠 [블랙],https://image.msscdn.net/thumbnails/images/goo...,97,\nLV.5 인텐시버\n2023.01.13\n\n\n\n\n\n,\n\n남성 · 164cm · 64kg\n\n,옷도 짱짱하고 좋아요 그리고 옷 색빠짐도 별로 없는 것 같아요,[M],사이즈 보통이에요,longs,1841331,[M],70.55,48.5,55.0,58.0


In [ ]:
# final_merge_df.to_csv(f"{link}final_merge_df.csv", index = False)